In [15]:
import itertools
import pandas as pd
from collections import Counter
import gc
from tqdm import tqdm
import bz2
import numpy as np
import re
import os

def from_bz2(train_file):
    with bz2.open(train_file, 'rt') as f:
        examples = []
        for s in tqdm(itertools.islice(f, 0, batch_size, step_size)):
            examples.append(s)
        jsonstr = '[' + ','.join(examples) + ']'
        df = pd.read_json(jsonstr, orient='list')
        return df

def from_json(train_file):
    with open(train_file, 'r', encoding='utf-8') as f:
        examples = []
        for s in itertools.islice(f, 0, batch_size, step_size):
            examples.append(s)
        jsonstr = '[' + ','.join(examples) + ']'
        df = pd.read_json(jsonstr, orient='list')
        return df

In [16]:
def from_bz2_minibatch(train_file):
    global df
    with bz2.open(train_file, 'rt') as f:
        mini_batch = int(1e4)
        print('mini batch', mini_batch)
        loops = batch_size // mini_batch
        remain = batch_size % mini_batch
        print('loops', loops, 'remain', remain)
        df = pd.DataFrame()
        for i in range(0, loops):
            if (i % 100 == 0):
                print('loop', i)
                print('gc', gc.collect())
            jsonstr = '[' 
            for s in tqdm(itertools.islice(f, 0, mini_batch, step_size)):
                jsonstr += s + ','
            jsonstr = jsonstr[:-1] + ']' 
            if df.empty:
                df = pd.read_json(jsonstr, orient='list')
            else:
                df1 = pd.read_json(jsonstr, orient='list')
                df=df.append(df1)
                del df1
            del jsonstr   
               
        jsonstr = '['        
        for s in tqdm(itertools.islice(f, 0, remain, step_size)):
            jsonstr += s + ','
        if len(jsonstr) > 1:
            jsonstr = jsonstr[:-1] + ']' 
            if df.empty:
                df = pd.read_json(jsonstr, orient='list')
            else:
                df1 = pd.read_json(jsonstr, orient='list')
                df=df.append(df1)
        else:
            print('No data in remain')
        del jsonstr   
        print('gc', gc.collect())
        return df

In [17]:
from sklearn.datasets import load_svmlight_file

In [18]:
def shuffle(data):
    X, y = data
    ind = np.arange(X.shape[0])
    for i in range(7):
        np.random.shuffle(ind)
    return X[ind], y[ind]

In [19]:
import boto
import boto.s3
import sys
from boto.s3.key import Key

conn = boto.connect_s3()

def get_pd_from_s3_csv(bucket, path, names=None, dtype=None):
    bucket = conn.get_bucket(bucket)
    regex = re.compile('^' + path)
    
    selected_files = filter(regex.match, [f.name for f in bucket])
    df = pd.DataFrame()
    for file in selected_files:
        print(file)
        if df.empty:
            df = pd.read_csv(bucket.get_key(file), delimiter=',',
                    names=names,
                    dtype = dtype)
        else:
            df.append(pd.read_csv(bucket.get_key(file), delimiter=',',
                    names= names,
                    dtype = dtype)) 
    return df
    

In [20]:
from sklearn.datasets import load_svmlight_files
import boto3
s3 = boto3.resource('s3')
def s3_files_to_local(bucket_name, path, output='/tmp/s3_files'):
    bucket = conn.get_bucket(bucket_name)
    regex = re.compile('^' + path)
    selected_files = filter(regex.match, [f.name for f in bucket])
    print(bucket)
    outputs = []
    for file in selected_files:
        print(file)
        file_name = file.split('/')[-1]
        file_path = os.path.join(output, file_name)
        s3.Bucket(bucket_name).download_file(file, file_path)
        outputs.append(file_path)
    return outputs 

In [28]:
def get_libsvms_from_s3(bucket_name, path, n_features=None):
    files = s3_files_to_local(bucket_name, path)
    x=load_svmlight_files(files, zero_based=True, n_features=n_features)
    return x
    